In [ ]:
"""
Notebook basics.

This notebook shows you the basic steps for loading and displaying data
from checklists and observations. It assumes you have completed he steps in
the "Getting Started" section of the README, and you have an eBird API key.
"""

from django.conf import settings
from django.template.loader import render_to_string

from faker import Faker
from IPython.display import display, HTML

from ebird.checklists.loaders import APILoader, SpeciesLoader
from ebird.checklists.models import Checklist, Observation

# Randomly choose a country to load checklists for.
region = Faker().country_code()

In [ ]:
# Load the complete eBird taxonomy into the Species table
loader = SpeciesLoader(settings.EBIRD_API_KEY)
loader.load()

In [ ]:
# Load the ten most recent checklists from the eBird API.
loader = APILoader(settings.EBIRD_API_KEY)
loader.recent(region, limit=10)

In [ ]:
# Show all the checklists just loaded.
checklists = Checklist.objects.filter(location__country_code=region)

table = render_to_string("checklists.html", {"checklists": checklists})
display(HTML(table))

In [ ]:
# Show the highest count for each species seen
# Rather than build a complex query to group the observations by species
# and calculate the highest count, we cheat by using the {% ifchanged %}
# tag in the template to display the observation each time the species
# changes. Sorting by count descending means this record is the one where
# the highest count was recorded.
observations = Observation.objects.filter(location__country_code=region).order_by("species__taxon_order", "-count")

table = render_to_string("highest-counts.html", {"observations": observations})
display(HTML(table))